In [0]:
import os

# Set the environment variable for Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/dbfs/FileStore/tables/promising-silo-374208-8b3d9c4e46c7.json"


Using service account to access gcs from databricks

In [0]:
print(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS'))

/dbfs/FileStore/tables/promising-silo-374208-8b3d9c4e46c7.json


Mounting the data from cloud storage

In [0]:
dbutils.fs.unmount("/mnt/gcs-data")

/mnt/gcs-data has been unmounted.


True

In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserved', encryptionType='')]

In [0]:
dbutils.fs.ls("/FileStore/tables/")

[FileInfo(path='dbfs:/FileStore/tables/promising_silo_374208_76133b3d97e6-1.json', name='promising_silo_374208_76133b3d97e6-1.json', size=2395, modificationTime=1733089073002),
 FileInfo(path='dbfs:/FileStore/tables/promising_silo_374208_76133b3d97e6-2.json', name='promising_silo_374208_76133b3d97e6-2.json', size=2395, modificationTime=1733089356299),
 FileInfo(path='dbfs:/FileStore/tables/promising_silo_374208_76133b3d97e6-3.json', name='promising_silo_374208_76133b3d97e6-3.json', size=2395, modificationTime=1733091005297),
 FileInfo(path='dbfs:/FileStore/tables/promising_silo_374208_76133b3d97e6.json', name='promising_silo_374208_76133b3d97e6.json', size=2395, modificationTime=1733089048426),
 FileInfo(path='dbfs:/FileStore/tables/promising_silo_374208_7f801d04cf6f.json', name='promising_silo_374208_7f801d04cf6f.json', size=2395, modificationTime=1733096982546),
 FileInfo(path='dbfs:/FileStore/tables/promising_silo_374208_8b3d9c4e46c7.json', name='promising_silo_374208_8b3d9c4e46c7.j

In [0]:
dbutils.fs.mount(
    source="gs://603-yelp-data",  
    mount_point="/mnt/gcs-data",  
    extra_configs={
        "google.cloud.auth.service.account.json.keyfile": "/dbfs/FileStore/tables/promising_silo_374208_8b3d9c4e46c7.json"
    }
)

True

In [0]:
dbutils.fs.ls("/mnt/gcs-data")

[FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/', name='processed_reviews.parquet/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gcs-data/yelp-data/', name='yelp-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gcs-data/yelp_academic_dataset_review.json', name='yelp_academic_dataset_review.json', size=5341868833, modificationTime=1733094816069)]

EDA analysis

In [0]:
yelp_reviews = spark.read.json("/mnt/gcs-data/yelp_academic_dataset_review.json")
yelp_reviews.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [0]:
yelp_reviews.show(5, truncate=False) 

+----------------------+----+-------------------+-----+----------------------+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+----------------------+
|business_id           |cool|date               |fu

In [0]:
# Print schema of the dataset
yelp_reviews.printSchema()

# Get the total number of rows
row_count = yelp_reviews.count()
print(f"Total number of rows: {row_count}")

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

Total number of rows: 6990280


In [0]:
# Drop duplicate reviews and remove entries with blank or irrelevant text
cleaned_reviews = yelp_reviews.dropDuplicates(["review_id"]).filter("text IS NOT NULL AND text != ''")

# Display summary of the cleaned dataset
cleaned_reviews.describe(["stars", "useful", "funny", "cool"]).show()

+-------+------------------+------------------+-------------------+------------------+
|summary|             stars|            useful|              funny|              cool|
+-------+------------------+------------------+-------------------+------------------+
|  count|           6990280|           6990280|            6990280|           6990280|
|   mean|  3.74858374771826|1.1846089140921394|0.32655959417934616|0.4986175088837643|
| stddev|1.4787045052556806| 3.253766966933353|  1.688729098554047| 2.172459820211185|
|    min|               1.0|                -1|                 -1|                -1|
|    max|               5.0|              1182|                792|               404|
+-------+------------------+------------------+-------------------+------------------+



In [0]:
from pyspark.sql.functions import col, isnan, when, count

# Count null or NaN values for each column
cleaned_reviews.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in cleaned_reviews.columns]).show()

+-----------+----+----+-----+---------+-----+----+------+-------+
|business_id|cool|date|funny|review_id|stars|text|useful|user_id|
+-----------+----+----+-----+---------+-----+----+------+-------+
|          0|   0|   0|    0|        0|    0|   0|     0|      0|
+-----------+----+----+-----+---------+-----+----+------+-------+



There are zero null values in the dataset


In [0]:
from pyspark.sql.functions import length

cleaned_reviews = cleaned_reviews.withColumn("review_length", length(col("text")))

In [0]:
from pyspark.sql.functions import year, month

cleaned_reviews = cleaned_reviews.withColumn("review_year", year(col("date"))) \
                       .withColumn("review_month", month(col("date")))

In [0]:
cleaned_reviews.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- review_year: integer (nullable = true)
 |-- review_month: integer (nullable = true)



In [0]:
cleaned_reviews = cleaned_reviews.withColumn(
    "rating_category",
    when(col("stars") >= 4, "high").when(col("stars") == 3, "medium").otherwise("low")
)

In [0]:
cleaned_reviews.groupBy("stars").count().orderBy("stars").show()

+-----+-------+
|stars|  count|
+-----+-------+
|  1.0|1069561|
|  2.0| 544240|
|  3.0| 691934|
|  4.0|1452918|
|  5.0|3231627|
+-----+-------+



In [0]:
cleaned_reviews.select("review_length").describe().show()

+-------+-----------------+
|summary|    review_length|
+-------+-----------------+
|  count|          6990280|
|   mean|567.7644364746477|
| stddev|527.2578085249031|
|    min|                1|
|    max|             5000|
+-------+-----------------+



In [0]:
cleaned_reviews.groupBy("review_year").count().orderBy("review_year").show()

+-----------+------+
|review_year| count|
+-----------+------+
|       2005|   854|
|       2006|  3853|
|       2007| 15363|
|       2008| 48226|
|       2009| 74387|
|       2010|138587|
|       2011|230813|
|       2012|286570|
|       2013|383950|
|       2014|522275|
|       2015|688415|
|       2016|758882|
|       2017|820048|
|       2018|906362|
|       2019|907284|
|       2020|554557|
|       2021|618189|
|       2022| 31665|
+-----------+------+



In [0]:
cleaned_reviews.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- review_year: integer (nullable = true)
 |-- review_month: integer (nullable = true)
 |-- rating_category: string (nullable = false)



After completion of processing storing the processed file in cloud storage

In [0]:
# Write the DataFrame to Parquet format and overwrite the existing file
cleaned_reviews.write.mode("overwrite").parquet("dbfs:/mnt/gcs-data/processed_reviews.parquet")


In [0]:
# Verify if the file is written to the location
dbutils.fs.ls("dbfs:/mnt/gcs-data/processed_reviews.parquet/")

[FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/_SUCCESS', name='_SUCCESS', size=0, modificationTime=1733179129393),
 FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/_committed_7367402482515525943', name='_committed_7367402482515525943', size=6791, modificationTime=1733178380649),
 FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/_committed_8403372472510479157', name='_committed_8403372472510479157', size=13568, modificationTime=1733179127633),
 FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/_started_7367402482515525943', name='_started_7367402482515525943', size=0, modificationTime=1733178362726),
 FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/_started_8403372472510479157', name='_started_8403372472510479157', size=0, modificationTime=1733179113982),
 FileInfo(path='dbfs:/mnt/gcs-data/processed_reviews.parquet/part-00000-tid-8403372472510479157-33e61a39-de86-47d9-866a-189c00403714-1759-1-c000.snappy.parquet', name='part-00

Success